<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.0.1"><span class="toc-item-num">1.0.1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Загрузка-стоп-слов-и-wordnet" data-toc-modified-id="Загрузка-стоп-слов-и-wordnet-1.0.2"><span class="toc-item-num">1.0.2&nbsp;&nbsp;</span>Загрузка стоп-слов и wordnet</a></span></li><li><span><a href="#Предобработка-и-лемматизация-текста" data-toc-modified-id="Предобработка-и-лемматизация-текста-1.0.3"><span class="toc-item-num">1.0.3&nbsp;&nbsp;</span>Предобработка и лемматизация текста</a></span></li><li><span><a href="#Токенизация-TF-IDF-и-разделение-на-выборки" data-toc-modified-id="Токенизация-TF-IDF-и-разделение-на-выборки-1.0.4"><span class="toc-item-num">1.0.4&nbsp;&nbsp;</span>Токенизация TF-IDF и разделение на выборки</a></span></li></ul></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Обучение-моделей-" data-toc-modified-id="Обучение-моделей--2.0.1"><span class="toc-item-num">2.0.1&nbsp;&nbsp;</span>Обучение моделей <br></a></span></li><li><span><a href="#Промежуточный-вывод-" data-toc-modified-id="Промежуточный-вывод--2.0.2"><span class="toc-item-num">2.0.2&nbsp;&nbsp;</span>Промежуточный вывод <br></a></span></li></ul></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span><ul class="toc-item"><li><span><a href="#Комментарий-студента-" data-toc-modified-id="Комментарий-студента--3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Комментарий студента <br></a></span></li></ul></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [9]:
import pandas as pd
import nltk
from nltk import WordNetLemmatizer, word_tokenize, pos_tag
from nltk.corpus import stopwords as nltk_stopwords, wordnet
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import re
from scipy.stats import randint, uniform

In [10]:
RANDOM_STATE = 42

## Подготовка

#### Загрузка данных

In [11]:
try:
    comments = pd.read_csv('../datasets/toxic_comments.csv')
except FileNotFoundError:
    comments = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

comments = comments.rename(columns={comments.columns[0]: 'index'})
comments = comments.set_index('index')

In [12]:
comments

,text,toxic
index,,
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159446,""":::::And for the second time of asking, when ...",0
159447,You should be ashamed of yourself \n\nThat is ...,0
159448,"Spitzer \n\nUmm, theres no actual article for ...",0


#### Загрузка стоп-слов и wordnet

In [13]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

stopwords = set(nltk_stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nikitakhavkin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nikitakhavkin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nikitakhavkin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/nikitakhavkin/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [14]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

#### Предобработка и лемматизация текста

In [15]:
def preprocess_text(text):
    # Удаляем все, кроме букв и пробелов
    text = re.sub(r'[^а-яА-Яa-zA-Z\s]', '', text)
    # Приводим к нижнему регистру
    text = text.lower()
    # Токенизация текста
    words = word_tokenize(text)
    # POS-тегирование
    pos_tags = pos_tag(words)
    # Удаляем стоп-слова и лемматизируем
    lemmatized_words = [
        lemmatizer.lemmatize(word, get_wordnet_pos(word))
        for word, _ in pos_tags
        if word not in stopwords
    ]
    # Возвращаем предобработанный текст
    return ' '.join(lemmatized_words)

In [16]:
comments['text'] = comments['text'].apply(preprocess_text)

#### Токенизация TF-IDF и разделение на выборки

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    comments['text'], comments['toxic'], test_size=0.4, random_state=RANDOM_STATE
)

In [18]:
vectorizer = TfidfVectorizer(stop_words=list(stopwords))

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

## Обучение

#### Обучение моделей <br>
Обучим 3 модели: логистическую регрессию, случайный лес и градиентный бустинг

In [19]:
models = [
    ("Logistic Regression", LogisticRegression(random_state=RANDOM_STATE)),
    ("Random Forest", RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced')),
    ("Gradient Boosting", GradientBoostingClassifier(random_state=RANDOM_STATE, n_estimators=50, learning_rate=0.2))
]

In [20]:
param_distributions = {
    "Logistic Regression": {
        'C': uniform(0.01, 1),
        'penalty': ['l1', 'l2', 'elasticnet'],
        'solver': ['saga']
    },
    "Random Forest": {
        'n_estimators': randint(100, 500),
        'max_depth': randint(5, 50),
        'min_samples_split': randint(2, 10),
        'min_samples_leaf': randint(1, 10)
    },
    "Gradient Boosting": {
        'n_estimators': randint(50, 200),
        'learning_rate': uniform(0.01, 0.3),
        'max_depth': randint(3, 10)
    }
}

In [21]:
cv = 3
best_model_name = None
best_model = None
best_score = 0

Цикл для обучения моделей и вывода метрики F1

In [22]:
for model_name, model in models:
    search = RandomizedSearchCV(
        model,
        param_distributions=param_distributions[model_name],
        scoring='f1',
        cv=cv,
        n_iter=20,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    search.fit(X_train, y_train)

    mean_score = search.best_score_
    print(f"Best F1-score for {model_name} (CV={cv}): {mean_score:.4f}")
    print(f"Best parameters for {model_name}: {search.best_params_}")


    if mean_score > best_score:
        best_score = mean_score
        best_model_name = model_name
        best_model = search.best_estimator_

print(f"Best model: {best_model_name} with F1-score: {best_score:.4f}")

/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
21 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
21 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py", line 1204, in fit
    raise 

Best F1-score for Logistic Regression (CV=3): 0.7485
Best parameters for Logistic Regression: {'C': 0.9932308858067882, 'penalty': 'l1', 'solver': 'saga'}


KeyboardInterrupt: 

Оценка лучшей модели на тестовой выборке

In [100]:
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred)

print(f"Best model: {best_model_name}")
print(f"Test F1-score for {best_model_name}: {f1:.4f}")

Best model: Logistic Regression
Test F1-score for Logistic Regression: 0.7629


#### Промежуточный вывод <br>
Лучшая модель - логистическая регрессия с F1-score на тестовой выборке 0.7629 <br>
F1-score больше 0.75, цель достигнута

## Выводы

В ходе исследования были выполнены следующие шаги:
1. Загружены данные
2. Загружены стоп-слова
3. Проведена предобработка данных (токенизация)
4. Обучены 3 модели: логистическая регрессия, случайный лес и градиентный бустинг
5. Проведена оценка качества моделей по метрике F1
6. Вывод: лучшее значение метрики показала модель логистической регрессии - 0.7629